In [1]:
import pymysql
import baostock as bs
import pandas as pd
import time

In [2]:
col_d = 'date,code,open,high,low,close,preclose,adjustflag,volume,amount,tradestatus,turn,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST'
col_w_m = 'date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg'

In [3]:
def baostock_stock(code,col,sd,ed,fq,ajf):
    #### 登陆baostock系统 ####
    lg = bs.login()
    # 显示登陆返回信息
    # lg.error_code：错误代码，当为“0”时表示成功，当为非0时表示失败
    print('login respond error_code:'+lg.error_code)
    # lg.error_msg：错误信息，对错误的详细解释
    print('login respond  error_msg:'+lg.error_msg)
    
    #### 获取历史K线数据 ####
    # frequency="d"取日k线，adjustflag="3"默认不复权
    # 复权状态(1：后复权， 2：前复权，3：不复权），指数自动为3，不复权
    print('正在获取股票数据：'+code)
    rs = bs.query_history_k_data_plus(code,col,
         start_date=sd, end_date=ed, frequency=fq, adjustflag=ajf)
    print('query_history_k_data_plus respond error_code:'+rs.error_code)
    print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)
    
    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    result['frequency'] = fq
    
    #### 登出系统 ####
    bs.logout()
    return result

In [4]:
def database_stock(code,stock,fq,ajf):
    #### 连接database ####
    # conn = pymysql.connect(host=“你的数据库地址”, user=“用户名”,password=“密码”,database=“数据库名”,charset=“utf8”)
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456' ,charset='utf8')
    # 使用cursor()方法获取操作游标，cursor()得到一个可以执行SQL语句的光标对象
    cursor = conn.cursor()
    #### 选择数据库：stock ####
    Select_DataBase = "use stock;"
    cursor.execute(Select_DataBase)
    table_name = code.replace(".", '')
    
    print('正在创建数据表：'+table_name)
    # 定义要执行的SQL语句，预处理语句创建表
    try:
        sql_create = """create table %s"""% table_name +""" (
                        id int auto_increment,
                        date VARCHAR(20),
                        code VARCHAR(20),
                        open VARCHAR(20),
                        high VARCHAR(20),
                        low VARCHAR(20),
                        close VARCHAR(20),
                        preclose VARCHAR(20),
                        adjustflag VARCHAR(10),
                        volume VARCHAR(20),
                        amount VARCHAR(20),
                        tradestatus VARCHAR(10),
                        turn VARCHAR(20),
                        pctChg VARCHAR(20),
                        peTTM VARCHAR(20),
                        pbMRQ VARCHAR(20),
                        psTTM VARCHAR(20),
                        pcfNcfTTM VARCHAR(20),
                        isST VARCHAR(10),
                        frequency VARCHAR(10),
                        PRIMARY KEY (id),
                        UNIQUE KEY (date,code,adjustflag,frequency)
                        )"""
        # 执行SQL语句
        cursor.execute(sql_create)
    except:
        print(table_name+'数据表已存在！')
        
    sql_insert_d = ''
    sql_insert_w_m = ''
    length = len(stock)
    print('数据量：', length)
    if fq == 'd':
        for i in range(0, length):
            record = tuple(stock.loc[i])
            if sql_insert_d == '':
                #插入数据语句
                sql_insert_d = """INSERT INTO  %s"""% table_name +""" (date,code,open,high,low,close,preclose,
                                  adjustflag,volume,amount,tradestatus,turn,
                                  pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST,frequency) 
                                  VALUES ('%s','%s','%s','%s','%s','%s','%s',
                                  '%s','%s','%s','%s','%s',
                                  '%s','%s','%s','%s','%s','%s','%s')""" % record
            else:
                sql_insert_d = sql_insert_d + """,('%s','%s','%s','%s','%s','%s','%s',
                                                   '%s','%s','%s','%s','%s',
                                                   '%s','%s','%s','%s','%s','%s','%s')""" % record
        
        try:
            # 执行sql语句
            cursor.execute(sql_insert_d)
            # 提交到数据库执行
            conn.commit()
        except:
            # Rollback in case there is any error
            conn.rollback()
            
        if ajf == '1':
            print('【日线】【后复权[1]】数据成功存入数据库！\n')
        elif ajf == '2':
            print('【日线】【前复权[2]】数据成功存入数据库！\n')
        elif ajf == '3':
            print('【日线】【不复权[3]】数据成功存入数据库！\n')
    else:
        for i in range(0, length):
            record = tuple(stock.loc[i])
            if sql_insert_w_m == '':
                #插入数据语句
                sql_insert_w_m = """INSERT INTO  %s"""% table_name +""" (date,code,open,high,low,close,
                                    volume,amount,adjustflag,turn,pctChg,frequency) 
                                    VALUES ('%s','%s','%s','%s','%s','%s',
                                    '%s','%s','%s','%s','%s','%s')""" % record
            else:
                sql_insert_w_m = sql_insert_w_m + """,('%s','%s','%s','%s','%s','%s',
                                                       '%s','%s','%s','%s','%s','%s')""" % record
        try:
            # 执行sql语句
            cursor.execute(sql_insert_w_m)
            # 提交到数据库执行
            conn.commit()
        except:
            # Rollback in case there is any error
            conn.rollback()
            
        if fq == 'w':
            if ajf == '1':
                print('【周线】【后复权[1]】数据成功存入数据库！\n')
            elif ajf == '2':
                print('【周线】【前复权[2]】数据成功存入数据库！\n')
            elif ajf == '3':
                print('【周线】【不复权[3]】数据成功存入数据库！\n')
        elif fq == 'm':
            if ajf == '1':
                print('【月线】【后复权[1]】数据成功存入数据库！\n')
            elif ajf == '2':
                print('【月线】【前复权[2]】数据成功存入数据库！\n')
            elif ajf == '3':
                print('【月线】【不复权[3]】数据成功存入数据库！\n')
    # 关闭光标对象
    cursor.close()
    # 关闭数据库连接
    conn.close()

In [5]:
#### 登陆系统 ####
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)
# 获取沪深300成分股
rs = bs.query_hs300_stocks()
print('query_hs300 error_code:'+rs.error_code)
print('query_hs300  error_msg:'+rs.error_msg)
# 打印结果集
hs300_stocks = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    hs300_stocks.append(rs.get_row_data())
hs300 = pd.DataFrame(hs300_stocks, columns=rs.fields)
#### 登出系统 ####
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
query_hs300 error_code:0
query_hs300  error_msg:success
logout success!


In [6]:
code_list = tuple(hs300['code'])
# code_list = ['sz.002819']
sd = '2006-01-01'
ed = time.strftime('%Y-%m-%d',time.localtime(time.time()))
frequency_list = ['d','w','m']
adjustflag_list = ['1','2','3']
print(ed)

2019-09-21


In [7]:
def format_seconds(s):
    s = int(s)
    hh = s // 3600
    s = s % 3600
    mm = s // 60
    s = s % 60
    return str(hh)+':'+str(mm)+':'+str(s)

In [8]:
start_time = time.time()

In [9]:
for code in code_list:
    for fq in frequency_list:
        if fq == 'd':
            col = col_d
        else:
            col = col_w_m
        for ajf in adjustflag_list:
            stock_data = baostock_stock(code,col,sd,ed,fq,ajf)
            database_stock(code,stock_data,fq,ajf)

print('ALL Success!')

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600000
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600000
数据量： 3337
【日线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600000
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600000
sh600000数据表已存在！
数据量： 3337
【日线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600000
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600000
sh600000数据表已存在！
数据量： 3337
【日线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600000
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg

【日线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600010
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600010
sh600010数据表已存在！
数据量： 698
【周线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600010
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600010
sh600010数据表已存在！
数据量： 698
【周线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600010
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600010
sh600010数据表已存在！
数据量： 698
【周线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600010
query_history_k_data_plus respond error_code:0
query_h

【周线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600016
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600016
sh600016数据表已存在！
数据量： 164
【月线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600016
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600016
sh600016数据表已存在！
数据量： 164
【月线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600016
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600016
sh600016数据表已存在！
数据量： 164
【月线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600018
query_history_k_data_plus respond error_code:0
query_h

正在创建数据表：sh600023
sh600023数据表已存在！
数据量： 69
【月线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600025
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600025
数据量： 431
【日线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600025
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600025
sh600025数据表已存在！
数据量： 431
【日线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600025
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600025
sh600025数据表已存在！
数据量： 431
【日线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600025
query_history_k_data_plus res

【日线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600029
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600029
sh600029数据表已存在！
数据量： 698
【周线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600029
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600029
sh600029数据表已存在！
数据量： 698
【周线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600029
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sh600029
sh600029数据表已存在！
数据量： 698
【周线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sh.600029
query_history_k_data_plus respond error_code:0
query_h

...

【日线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300059
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300059
sz300059数据表已存在！
数据量： 2315
【日线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300059
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300059
sz300059数据表已存在！
数据量： 2315
【日线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300059
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300059
sz300059数据表已存在！
数据量： 487
【周线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300059
query_history_k_data_plus respond error_code:0
query

【周线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300122
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300122
sz300122数据表已存在！
数据量： 459
【周线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300122
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300122
sz300122数据表已存在！
数据量： 459
【周线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300122
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300122
sz300122数据表已存在！
数据量： 108
【月线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300122
query_history_k_data_plus respond error_code:0
query_h

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300142
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300142
sz300142数据表已存在！
数据量： 106
【月线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300142
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300142
sz300142数据表已存在！
数据量： 106
【月线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300144
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300144
数据量： 2139
【日线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300144
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:s

【日线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300408
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300408
sz300408数据表已存在！
数据量： 1173
【日线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300408
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300408
sz300408数据表已存在！
数据量： 1173
【日线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300408
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300408
sz300408数据表已存在！
数据量： 245
【周线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300408
query_history_k_data_plus respond error_code:0
query

【周线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300498
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300498
sz300498数据表已存在！
数据量： 198
【周线】【前复权[2]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300498
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300498
sz300498数据表已存在！
数据量： 198
【周线】【不复权[3]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300498
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!
正在创建数据表：sz300498
sz300498数据表已存在！
数据量： 46
【月线】【后复权[1]】数据成功存入数据库！

login success!
login respond error_code:0
login respond  error_msg:success
正在获取股票数据：sz.300498
query_history_k_data_plus respond error_code:0
query_hi

In [10]:
time_used = time.time()-start_time
format_seconds(time_used)

'0:21:18'